<a href="https://colab.research.google.com/github/GTRe5/AI/blob/main/523H0135_lec04_UninformedSearch_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Students refer to pseudo codes of BFS, DFS, UCS, DLS, and IDS in [this link](https://drive.google.com/file/d/1q2LtrRCfemfiqyhfxNMcVJ3alvLh_pdV/view?usp=share_link) to implement the corresponding classes in TODO 1 - 5. \
Students can add supporting attributes and methods to the five classes of search strategies as needed.

# Libraries

In [96]:
import os
import heapq

# Graph class

In [97]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...

        # input.txt
        7 8
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for line in g:
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))

In [98]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Search Strategies

In [100]:
class SearchStrategy:
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    return expanded, path

In [101]:
class BFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 1
    node = src
    if node == dst :
      return [node], [node]

    frontier = [node] #FIFO queue
    explored = set()
    parent = {node : None}

    while frontier :
      node = frontier.pop(0) # Pop from queue (FIFO)
      expanded.append(node)
      explored.add(node)


      if node == dst :  # If it find goal, then stop the loop
        break

      # Explore all neighbors of the current node
      if node in g.AL :
        for neighbor, _ in g.AL[node] :  # Get all node's neighbors
          if neighbor not in explored and neighbor not in frontier :
            parent[neighbor] = node
            frontier.append(neighbor)
    """
     for key, value in parent.items():
       print(f"{key} : parent = {value}")
    """

    # Reconstruct time hehe
    if dst in parent :
      neighbor = dst
      while neighbor is not None :
        path.append(neighbor)
        neighbor = parent[neighbor]
      path.reverse()

    return expanded, path

In [102]:
class DFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 2
    node = src
    if node == dst :
      return [node], [node]

    frontier = [node]  # LIFO stack
    explored = set()
    parent = {node : None}

    while frontier :
      node = frontier.pop()  # Pop from stack (LIFO)
      if node in explored:
        continue  # Skip already visited nodes

      expanded.append(node)
      explored.add(node)

      if node == dst:  # If it find goal, then stop the loop
        break

      # Explore all neighbors of the current node
      if node in g.AL:
        for neighbor, _ in reversed(g.AL[node]) :  # Reverse to match recursive DFS order
          if neighbor not in explored :
            parent[neighbor] = node
            frontier.append(neighbor)  # Add to stack (LIFO)

    # Reconstruct time ehehe
    if dst in parent :
      neighbor = dst
      while neighbor is not None :
        path.append(neighbor)
        neighbor = parent[neighbor]
      path.reverse()

    return expanded, path

In [103]:
class UCS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 3
    frontier = [(0, src)] # Priority queue
    explore = {src : 0} # Similar with the shortest cost of each node
    parent = {src : None}

    while frontier :
      cost, node = heapq.heappop(frontier)

      if node in expanded:
        continue  # Skip nodes already expanded

      expanded.append(node)

      if node == dst :  # Stop if we reach the goal
        break

      # Explore all neighbors of the current node
      if node in g.AL:
        for neighbor, weight in g.AL[node] :
          new_cost = cost + weight  # Compute new path cost

          # If the neighbor is unvisited OR we found a cheaper path to it
          if neighbor not in explore or new_cost < explore[neighbor] :
            explore[neighbor] = new_cost
            parent[neighbor] = node
            heapq.heappush(frontier, (new_cost, neighbor))  # Push with priority

    # Reconstruct 3rd time
    if dst in parent :
      node = dst
      while node is not None :
        path.append(node)
        node = parent[node]
      path.reverse()


    return expanded, path

In [104]:
class DLS(SearchStrategy):
  def __init__(self, LIM: int):
    self.LIM = LIM

  def recursive_dls(self, g: Graph,
                    node: int, dst: int,depth: int,
                    expanded: list, path: list, parent: dict) -> bool :
    if depth < 0 or depth > self. LIM :
      return False  # Cutoff condition

    expanded.append(node)

    if node == dst :  # Stop if we reach the goal
      path.append(node)
      return True

    if node in g.AL:  # Expand children
      for child, _ in g.AL[node] :
          if child not in expanded :
              parent[child] = node
              if self.recursive_dls(g, child, dst, depth - 1, expanded, path, parent) :
                  path.append(node)  # Backtrack to reconstruct the path
                  return True

    return False

  def search(self, g: Graph, src: int, dst: int) -> tuple :
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 4
    parent = {src : None}
    if self.recursive_dls(g, src, dst, self.LIM, expanded, path, parent) :
      path.reverse()  # Anyway reconstruction timeeeee
    else :
      path = []

    return expanded, path

In [105]:
class IDS(SearchStrategy):
  def __init__(self, MAX_LIM: int):
    self.MAX_LIM = MAX_LIM

  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 5
    for depth in range(self.MAX_LIM + 1) :
      dls = DLS(depth)  # Create a Depth-Limited Search (DLS) instance
      expanded, path = dls.search(g, src, dst)

      if path :  # If only path exists it will return right now
        return expanded, path

    return expanded, path

# Evaluation

In [106]:
bfs = BFS()
dfs = DFS()
ucs = UCS()
dls = DLS(LIM=3)
ids = IDS(MAX_LIM=5)

for stg in [bfs, dfs, ucs, dls, ids]:
  print(stg)
  expanded, path = stg.search(g, 0, g.V-1)
  print(expanded)
  print(path)




[0, 1, 2, 3, 4, 5, 6]
[0, 1, 3, 6]
[0, 1, 3, 5, 6]
[0, 1, 3, 6]
[0, 1, 2, 5, 4, 3, 6]
[0, 1, 4, 6]
[0, 1, 3, 5, 6]
[0, 1, 3, 6]
[0, 1, 3, 5, 6]
[0, 1, 3, 6]


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-lec04-UninformedSearch-HW.ipynb**
*   Finally, submit the file

